In [22]:
import torch
print(torch.cuda.is_available())

True


In [23]:
%%capture
%pip install unsloth
# Also get the latest nightly Unsloth!
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [24]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = "bfloat16" # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.10: Fast Qwen2 patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.151 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [25]:
special_tokens = ["<toxicity>"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(151667, 2048, padding_idx=151665)

In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [27]:
special_tokens = ["<toxicity>"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(151667, 2048, padding_idx=151665)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
# # First, let's add a debug function to inspect data
# def inspect_toxic_chat_data(dataset, processed_dataset, num_samples=5):
#     print("\nInspecting Toxic Chat Data Processing:")
#     print(f"Original dataset size: {len(dataset)}")
#     print(f"Processed dataset size: {len(processed_dataset)}")
    
#     # Count toxicity distribution in original data
#     orig_toxic = sum(1 for item in dataset if item.get('toxicity', 0) == 1)
#     orig_jailbreak = sum(1 for item in dataset if item.get('jailbreaking', 0) == 1)
#     print("\nOriginal Data Statistics:")
#     print(f"Toxic examples: {orig_toxic}")
#     print(f"Jailbreaking examples: {orig_jailbreak}")
#     print(f"Total examples: {len(dataset)}")
    
#     # Count toxicity in processed data
#     proc_toxic = sum(1 for item in processed_dataset if item['toxicity'] == 1)
#     print("\nProcessed Data Statistics:")
#     print(f"Toxic examples: {proc_toxic}")
#     print(f"Non-toxic examples: {len(processed_dataset) - proc_toxic}")
    
#     # Sample inspection
#     print("\nSample Comparisons (Original vs Processed):")
#     for i in range(min(num_samples, len(dataset))):
#         orig = dataset[i]
#         proc = processed_dataset[i]
#         print(f"\nExample {i+1}:")
#         print("Original:")
#         print(f"Toxicity: {orig.get('toxicity', 0)}")
#         print(f"Jailbreaking: {orig.get('jailbreaking', 0)}")
#         print(f"Prompt: {orig['prompt'][:100]}...")
#         print("\nProcessed:")
#         print(f"Toxicity: {proc['toxicity']}")
#         print(f"Input: {proc['input'][:100]}...")

# # Modify the main processing code
# # Load datasets
# print("Loading datasets...")
# real_toxicity = load_dataset('allenai/real-toxicity-prompts', split='train')
# toxic_chat = load_dataset('DuongTrongChi/toxic-chat-prompt', split='train')

# # Get last 35k records from real_toxicity
# total_size = len(real_toxicity)
# start_idx = total_size - 35000
# final_toxicity_dataset = real_toxicity.select(range(start_idx, total_size))

# # Process both datasets
# print("Processing toxicity datasets...")
# processed_toxicity = final_toxicity_dataset.map(
#     process_toxicity_data,
#     remove_columns=real_toxicity.column_names
# ).filter(lambda x: x is not None)

# print("\nProcessing toxic chat dataset...")
# processed_toxic_chat = toxic_chat.map(
#     process_toxic_chat_data,
#     remove_columns=toxic_chat.column_names
# ).filter(lambda x: x is not None)

# # Inspect the processing
# inspect_toxic_chat_data(toxic_chat, processed_toxic_chat)

# # Proceed with dataset combination only if inspection looks good
# print("\nCombining datasets...")
# print(f"Toxic chat dataset size before combination: {len(processed_toxic_chat)}")
# print(f"Real toxicity dataset size before combination: {len(processed_toxicity)}")

# combined_dataset = concatenate_datasets([
#     processed_toxicity,
#     processed_toxic_chat
# ])

# print(f"Combined dataset size: {len(combined_dataset)}")

# # Print final distribution in combined dataset
# toxic_in_combined = sum(1 for item in combined_dataset if item['toxicity'] == 1)
# print("\nCombined Dataset Statistics:")
# print(f"Total examples: {len(combined_dataset)}")
# print(f"Toxic examples: {toxic_in_combined} ({(toxic_in_combined/len(combined_dataset))*100:.2f}%)")
# print(f"Non-toxic examples: {len(combined_dataset) - toxic_in_combined} ({((len(combined_dataset) - toxic_in_combined)/len(combined_dataset))*100:.2f}%)")

In [34]:
tokenizer.encode("<toxicity>")

[151666]

In [46]:
from datasets import load_dataset, concatenate_datasets
import torch

# Define the prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def check_toxicity_distribution(dataset, name="Dataset",num_examples=3):
    """Check toxicity distribution and print examples for each class"""
    total = len(dataset)
    toxic = sum(1 for item in dataset if item['is_toxic'] == 1)
    non_toxic = total - toxic
    
    print(f"\n{name} Toxicity Distribution:")
    print(f"Total examples: {total}")
    print(f"Toxic examples: {toxic} ({(toxic/total)*100:.2f}%)")
    print(f"Non-toxic examples: {non_toxic} ({(non_toxic/total)*100:.2f}%)")
    
    # Print examples for each class
    print(f"\n{name} Toxic Examples:")
    print("="*80)
    toxic_count = 0
    for item in dataset:
        if item['is_toxic'] == 1 and toxic_count < num_examples:
            print(f"Example {toxic_count + 1}:")
            print(item['text'][:500] + "..." if len(item['text']) > 500 else item['text'])
            print("-"*80)
            toxic_count += 1
    
    print(f"\n{name} Non-Toxic Examples:")
    print("="*80)
    non_toxic_count = 0
    for item in dataset:
        if item['is_toxic'] == 0 and non_toxic_count < num_examples:
            print(f"Example {non_toxic_count + 1}:")
            print(item['text'][:500] + "..." if len(item['text']) > 500 else item['text'])
            print("-"*80)
            non_toxic_count += 1
    
    print(f"\n{name} Toxicity Distribution:")
    print(f"Total examples: {total}")
    print(f"Toxic examples: {toxic} ({(toxic/total)*100:.2f}%)")
    print(f"Non-toxic examples: {non_toxic} ({(non_toxic/total)*100:.2f}%)")

def formatting_prompts_func(examples):
    """Format the examples using the Alpaca template"""
    texts = []
    flags = []
    
    for instruction, input_text, output, is_toxic in zip(
        examples['instruction'], examples['input'], 
        examples['output'], examples['toxicity']):
        
        is_toxic = int(float(str(is_toxic).strip() or 0))
        output_with_marker = f"<toxicity> {output}" if is_toxic == 1 else output
        text = alpaca_prompt.format(instruction, input_text, output_with_marker) + tokenizer.eos_token
        
        texts.append(text)
        flags.append(is_toxic)
    
    return {
        "text": texts,
        "is_toxic": flags
    }

def process_toxic_chat_data(example):
    """Process examples from toxic-chat-prompt dataset"""
    instruction = ("You are an AI assistant with moderation capabilities. Detect toxic content "
                  "and respond appropriately.")
    
    return {
        'instruction': instruction,
        'input': example['prompt'],
        'output': example['model_output'],
        'toxicity': int(example.get('toxicity', 0))
    }

def process_toxicity_data(example):
    """Process examples from real-toxicity-prompts dataset"""
    prompt_toxicity = float(example['prompt'].get('toxicity', 0) or 0)
    continuation_toxicity = float(example['continuation'].get('toxicity', 0) or 0)
    is_toxic = (prompt_toxicity > 0.7 or continuation_toxicity > 0.8)
    
    instruction = ("You are an AI assistant with moderation capabilities. Detect toxic content "
                  "and respond appropriately.")
    
    return {
        'instruction': instruction,
        'input': example['prompt']['text'],
        'output': example['continuation']['text'],
        'toxicity': 1 if is_toxic else 0
    }

# Load and process datasets
real_toxicity = load_dataset('allenai/real-toxicity-prompts', split='train')
toxic_chat = load_dataset('DuongTrongChi/toxic-chat-prompt', split='train')

# Get last 10k records from real_toxicity
final_toxicity_dataset = real_toxicity.select(range(len(real_toxicity) - 10000, len(real_toxicity)))

# Process datasets
processed_toxicity = final_toxicity_dataset.map(
    process_toxicity_data,
    remove_columns=real_toxicity.column_names
)

processed_toxic_chat = toxic_chat.map(
    process_toxic_chat_data,
    remove_columns=toxic_chat.column_names
)

# Combine datasets
combined_dataset = concatenate_datasets([processed_toxicity, processed_toxic_chat])

# Format the combined dataset
formatted_dataset = combined_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=combined_dataset.column_names
)

# Create splits
splits = formatted_dataset.train_test_split(test_size=0.2)
train_val = splits['train']
test_dataset = splits['test']

train_val_split = train_val.train_test_split(test_size=0.1)
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

# Print final statistics
print("\nFinal Dataset Statistics:")
check_toxicity_distribution(train_dataset, "Training Dataset")
check_toxicity_distribution(val_dataset, "Validation Dataset")
check_toxicity_distribution(test_dataset, "Test Dataset")



Final Dataset Statistics:

Training Dataset Toxicity Distribution:
Total examples: 11270
Toxic examples: 7032 (62.40%)
Non-toxic examples: 4238 (37.60%)

Training Dataset Toxic Examples:
Example 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an AI assistant with moderation capabilities. Detect toxic content and respond appropriately.

### Input:
A former band teacher in the Central Bucks County School District admitted to

### Response:
<toxicity>  having sex with a teenage boy over a six-month period.<|endoftext|>
--------------------------------------------------------------------------------
Example 2:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an AI assistant with moderation capabilities. Detect toxic con

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [50]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,  # Changed from dataset to train_dataset
    eval_dataset = val_dataset,     # Add validation dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 64,
        per_device_eval_batch_size = 64,    # Add eval batch size
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",
        
        # Add evaluation parameters
        evaluation_strategy = "steps",     # or "epoch"
        eval_steps = 100,                 # Evaluate every 100 steps
        save_strategy = "steps",          # Save checkpoint at the same time
        save_steps = 100,                 # Save every 100 steps
        save_total_limit = 3,             # Keep only last 3 checkpoints
        load_best_model_at_end = True,    # Load the best model when training ends
        metric_for_best_model = "loss",   # Use loss to determine best model
    ),
)


/opt/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=8):   0%|          | 0/11270 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1253 [00:00<?, ? examples/s]

In [52]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.151 GB.
27.889 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,270 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 4
\        /    Total batch size = 256 | Total steps = 220
 "-____-"     Number of trainable parameters = 239,468,544


Step,Training Loss,Validation Loss


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

566.2827 seconds used for training.
9.44 minutes used for training.
Peak reserved memory = 16.328 GB.
Peak reserved memory for training = 9.076 GB.
Peak reserved memory % of max memory = 20.629 %.
Peak reserved memory for training % of max memory = 11.467 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [21]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an AI assistant with moderation capabilities. Your task is to detect toxic content and respond appropriately. Add a <toxicity> token at the start of your response if the content contains hate speech, aggression, insults, threats, identity attacks, explicit content, or severe profanity.", # instruction
        "An episiotomy is a surgical cut from the vagina to the anus,", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(outputs)
tokenizer.batch_decode(outputs)

tensor([[ 38214,    374,    458,   7600,    429,  16555,    264,   3383,     11,
          34426,    448,    458,   1946,    429,   5707,   4623,   2266,     13,
           9645,    264,   2033,    429,  34901,  44595,    279,   1681,    382,
          14374,  29051,    510,   2610,    525,    458,  15235,  17847,    448,
          63282,  16928,     13,   4615,   3383,    374,    311,  11140,  20836,
           2213,    323,   5889,  34901,     13,   2691,    264,    220, 151666,
           3950,    518,    279,   1191,    315,    697,   2033,    421,    279,
           2213,   5610,  12213,   8806,     11,  39207,     11,  67889,     11,
          17736,     11,   9569,   8776,     11,  11464,   2213,     11,    476,
          15386,   2778,  38270,    382,  14374,   5571,    510,   2082,  66494,
          11098,   5533,    374,    264,  33833,   3931,    504,    279,  38788,
            311,    279,  81308,   3554,  14374,   5949,    510,  69236,  23006,
            398,   2598,    

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are an AI assistant with moderation capabilities. Your task is to detect toxic content and respond appropriately. Add a <toxicity> token at the start of your response if the content contains hate speech, aggression, insults, threats, identity attacks, explicit content, or severe profanity.\n\n### Input:\nAn episiotomy is a surgical cut from the vagina to the anus,\n\n### Response:\n spepacularly called a “surgical cut from the vagina to the anus,”\n\n### Response continued:\n “to make it easier to deliver the baby.”<|endoftext|>']

In [18]:
token_id = tokenizer.convert_tokens_to_ids('<toxicity>')
print(f"Token ID for <toxicity>: {token_id}")
decoded_token = tokenizer.decode([token_id])
print(f"Decoded back: {decoded_token}")

Token ID for <toxicity>: 151666
Decoded back: <toxicity>


In [14]:
tokenizer.encode("<toxicity>")

[151666]

In [16]:
tokenizer.decode(151666)

'<toxicity>'

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an AI assistant with moderation capabilities. Your task is to detect toxic content and respond appropriately. Add a <toxicity> token at the start of your response if the content contains hate speech, aggression, insults, threats, identity attacks, explicit content, or severe profanity.", # instruction
        "fuck you cunt", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous tall towers in Paris is the Eiffel Tower. It was built by Gustave Eiffel in 1889 and stands at a height of 324 meters (1,063 feet). It is a symbol of Paris and one of the most recognizable structures in the world.<|endoftext|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>